## 금주 목표
### 이번 시즌 팀 순위 시각화

In [62]:
import natsort as natsort
import pandas as pd
import os
import natsort
#from IPython.core.display_functions import display

In [63]:
for month in range(4, 10):
    url = "https://sports.news.naver.com/kbaseball/schedule/index?&month={}&year=2022&teamCode=".format(month)
    df = pd.read_html(url)
    for i in range(1, len(df)-1): #제일 위에 column 목차가 df단위로 떨어져서 삭제하는걸로 함, 그리고 마지막 df는 <맨위로>키 정보가 있는 테이블이라 범위에 넣지 않음.
        if df[i][2][0] == "프로야구 경기가 없습니다.":
            continue
        remain= df[i].loc[:, [0, 1, 2, 4, 5]]
        remain.rename(columns={0:"일자", 1:"시각", 2:"경기결과", 4:"중계사", 5:"구장"} , inplace=True)
        text = remain["일자"][0].split()[0]
        remain["일자"] = text
        remain.to_csv("./date/"+text+'.csv', header=True, encoding='UTF-8', index=False)
    print(month, "완료")

4 완료
5 완료
6 완료
7 완료
8 완료
9 완료


## DataFrame 에서 얻어낸 것
1. 네이버는 https 프로토콜을 사용하므로 ssl때문에 certification 관련 추가 작업이 필요했음
1-1. 따라서 terminal에 `pip install --upgrade certifi`를 추가적으로 작성함
1-2. 그리고 Application > python 3.10 > install certifi 진행
2. 네이버의 경우, 일정별로 깔끔하게 떨어져서 나옴
3. 다만, 데이터프레임의 개수가 많이 나타남

## 게임차 계산방법
```
팀 게임 승점 = (전체경기수 - 진행된 경기수)*0.5 + 승리 수 + 무승부*0.5
1위 승점 - 현재 팀 승점
```
- 패배를 왜 뺐나 했는데, 정말로 게임차 계산할 때 패배는 들어가지 않음.
- 실제로도 승리랑 무승부 가지고 판단함

In [69]:
start_data = {"LG":[0,0,0,0],
              "KIA":[0,0,0,0],
              "NC":[0,0,0,0],
              "SSG":[0,0,0,0],
              "한화":[0,0,0,0],
              "두산":[0,0,0,0],
              "키움":[0,0,0,0],
              "삼성":[0,0,0,0],
              "KT":[0,0,0,0],
              "롯데":[0,0,0,0]}
KBO_team = pd.DataFrame(start_data, index= ["경기수", "승리", "무승부","패배"])
KBO_team = KBO_team.T
KBO_team

,경기수,승리,무승부,패배
LG,0,0,0,0
KIA,0,0,0,0
NC,0,0,0,0
SSG,0,0,0,0
한화,0,0,0,0
두산,0,0,0,0
키움,0,0,0,0
삼성,0,0,0,0
KT,0,0,0,0
롯데,0,0,0,0


In [70]:
order_list = os.listdir("./date")
sorted_list = natsort.natsorted(order_list)
#sorted_list

In [66]:
for file in sorted_list:
    data = pd.read_csv("./date/" + file)
    for i in data.index:
        line = data.loc[i, :]

        if len(line[2].split(":")) == 1:
            continue

        away, home = line[2].split(":")
        home = home.split()
        away = away.split()
        h_score, h_team = home
        a_team, a_score = away
        h_score = int(h_score)
        a_score = int(a_score)

        if h_team == "나눔" or a_team == "나눔":
            continue #올스타전 제외

        #리팩토링은 항상 모두 완료하고나서 할 것
        if h_score > a_score: #홈팀 승리
            KBO_team.loc[h_team, ["승리"]] +=1
            KBO_team.loc[a_team, ["패배"]] +=1
        elif h_score < a_score:
            KBO_team.loc[a_team, ["승리"]] +=1
            KBO_team.loc[h_team, ["패배"]] +=1
        else:
            KBO_team.loc[h_team, ["무승부"]] +=1
            KBO_team.loc[a_team, ["무승부"]] +=1

        #경기수 구하기
        KBO_team["경기수"] = KBO_team["승리"] + KBO_team["패배"] + KBO_team["무승부"]

        #게임차 구하기
        KBO_team["승점"] = (144-KBO_team["경기수"])*0.5 + KBO_team["승리"] + KBO_team["무승부"]*0.5
        KBO_team.sort_values(by="승점", inplace=True, ascending=False)
        top = KBO_team["승점"][0]
        KBO_team["게임차"] = top - KBO_team["승점"]
        KBO_team.sort_values(by="승점", ascending=False, inplace=True)

        KBO_team["순위"] = [1,2,3,4,5,6,7,8,9,10]

        KBO_team.to_csv("./date_rank/"+file, header=True, encoding="UTF-8", index=True)
#원인찾았다. 이거 숫자가 문자열로 되어있어서 그랬음

In [67]:
KBO_team

,경기수,승리,무승부,패배,승점,게임차,순위
SSG,137,86,4,47,91.5,0.0,1
LG,133,82,2,49,88.5,3.0,2
키움,140,78,2,60,81.0,10.5,3
KT,136,75,2,59,80.0,11.5,4
KIA,136,65,1,70,69.5,22.0,5
NC,134,61,3,70,67.5,24.0,6
롯데,138,61,4,73,66.0,25.5,7
삼성,135,60,2,73,65.5,26.0,8
두산,134,56,2,76,62.0,29.5,9
한화,137,44,2,91,48.5,43.0,10


## 시각화 진행할 것

In [71]:
rank_list = os.listdir("./date_rank")
sorted_rank = natsort.natsorted(order_list)
#sorted_rank

### 정리
- 현재 필요한 정보: 순위 별 팀명, 일자
- 일단, 팀별로 일자/순위 정보 모두 종합해서 플롯 그려야함
- 그러면 자연스럽게 경기 없었던 일자에 누라정보 올 수 있어서 그거 채워야